In [134]:
# импорты
import nltk
import re
import pickle
from string import punctuation
import math
from tqdm import tqdm_notebook
import csv
import gensim
import random
from collections import Counter
from nltk.util import ngrams

import pymystem3
m = pymystem3.Mystem() #для использования лемматизации

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import pandas as pd
import numpy as np

'''from nltk.corpus import stopwords
stop_words = stopwords.words("russian")'''

'from nltk.corpus import stopwords\nstop_words = stopwords.words("russian")'

In [2]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

## Загрузка необходимых лексических баз:   
### словарей частот

In [3]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

In [4]:
# загружаем частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [5]:
bigrams = unpack('2grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
bigrams  = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in bigrams.items()}

In [6]:
list(bigrams.keys())[:10]

['дальнейший допрос',
 'оканчивать преступление',
 'развитие христианство',
 'и затонуть',
 'подразумевать она',
 'и однозначный',
 '2 введение',
 'желать услышать',
 'являться недооценка',
 'закрепляться по']

In [7]:
trigrams = unpack('3grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
trigrams = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in trigrams.items()}

In [6]:
list(trigrams.keys())[:10]

['в лифт я',
 'что сам русский',
 'почему ты думать',
 'фестиваль в кольмар',
 'твердость и прочность',
 'тот число руководитель',
 'судьба решать иначе',
 'мысль рождаться в',
 'мой учение то',
 'да только что']

## Загрузка необходимых лексических баз:   
### модели

In [8]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

Done!


In [ ]:
#model = model_loading('ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec')

## Загрузка необходимых лексических баз:   
### словаря

In [9]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

## Загрузка необходимых лексических баз:   
### минимумов

In [10]:
def load_minimum(typ=['A1','B1','B2'][0]):
    # всякие непонятные куски, портящие статистику по минимумам:
    to_remove = ['ми', 'вод', 'кий', 'ре','вая', 'ча', 'то быть', 'чка', 'стен']
    with open('{}.txt'.format(typ), 'r', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in file.read().lower().split('\n') if morph.parse(i)[0].normal_form not in to_remove] #делаем список лемм слов
        minimum = list(set(lemtokens)) #убираем повторы\
    return minimum

In [12]:
minimum = load_minimum('B1')

## Задание параметров способа определения сложности (частота или информационное содержание), идентификации слова как сложного (основываться только на минимуме/нет) и порога сложности

In [13]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf' # альтернатива: freq 
use_min=False

### Определение порога сложности на основе коэффициента информации (из униграмм) и минимумов

In [14]:
def make_threshold():
    # чем информационное содержание меньше, чем сложнее слово (с уменьшением дроби логарифм уменьшается тоже)
    det = sum(f for f in unigrams.values())+1
    # словарь цнф.содер и количества слов с таким коэф. в выбранном минимуме
    inf_coef = Counter([round(math.log((unigrams.get(w)+1)/det),1) for w in minimum if w in unigrams])
    # может быть очень низким (<17) для очень редких слов (матерь??) или опечаток, но в основном колеблется ок -9
    print(inf_coef.most_common(10))
    return np.median(list(inf_coef.keys()))

In [15]:
global_threshold = make_threshold()

[(-9.3, 68), (-9.4, 49), (-9.8, 47), (-9.7, 46), (-8.9, 45), (-9.6, 45), (-9.5, 44), (-10.2, 40), (-9.0, 37), (-9.2, 37)]


In [16]:
global_threshold

-9.6

## Таблица конверсии в UPoS из тэгов Mystem: в моделях используется первый, а при анализе текстов второй. Таблица конверсии из тегов Pymorphy2 - используется при назначении части речи словарным словам

In [17]:
# Таблица конверсии в UPoS из тэгов Mystem
# словарь, переводящий теги mystem в universal теги моделей
mystem_tags = {'A' : 'ADJ',
       'ADV' : 'ADV',
       'ADVPRO' : 'ADV',
       'ANUM' : 'ADJ',
       'APRO' : 'DET',
       'COM' : 'ADJ',
       'CONJ' : 'SCONJ',
       'INTJ' : 'INTJ',
       'NONLEX' : 'X',
       'NUM' : 'NUM',
       'PART' : 'PART',
       'PR' : 'ADP',
       'S' : 'NOUN',
       'SPRO' : 'PRON',
       'UNKN' : 'X',
       'V' : 'VERB',
       'X' : 'X',
      'PROPN' : 'PROPN'} #последних 2 тегов в майстеме нет, но они задаются в классе для слов в соответсвующими пометами

# словарь, переводящий теги пайморфи в universal теги моделей
pymorphy_tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}

## Основные классы: Токен (родительский), Сложное слово и Замена (дочерние)

In [24]:
class Token():
    def __init__(self, w):
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        self.av_similarity = None
        
        # три варианта инициализации: 
        ## из анализа текста, 
        ## из уже имеющегося объекта (для дочернего класса ComplexWord) 
        ## из строки
        
        if isinstance(w, dict): # если получили результат работы mystem
            
            self.text = w['text']  # сам токен
            self.len = len(w['text']) # его длина
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:
                self.lexem = gram[0]['lex']  # лемма
                
                if not self.named_entity(gram[0]):  # именованная сущность или нет
                    self.pos = self.pos_tag(gram[0]['gr'])  # часть речи
                else:
                    self.pos = 'PROPN' # universal tag for named entity - у майстема таких нет
                
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = 'X' # universal tag for unknown
            
            
        elif isinstance(w, Token): # для определения объектов дочернего класса ComplexWord
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
            
        
        elif isinstance(w, str): # если хотим как класс токен определить строку, полученную из словаря или модели
            self.text = w
            self.pos = None
            self.lexem = w
            self.len = len(w)
            self.num = None
            self.complexity = None
            
        
    # вытаскивает часть речи из разбора майстем
    def pos_tag(self,gram):
        if ',' in gram:
            gram = gram.split(',')[0]
        if '=' in gram:
            gram = gram.split('=')[0]
        return gram
        
    # определяет по тегам, является ли именованной сущностью
    def named_entity(self,gram):
        markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
        if any(m in gram['gr'] for m in markers.keys()):
            return True
        else:
            return False

    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lexem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        return self
    
    def is_complex(self, threshold = global_threshold, use_min = False):
        exceptions = ['_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions) and self.pos not in ['PROPN']:
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            else:
                if self.complexity < threshold:
                    return True
                else:
                    return False
        else:
            return False
    
    def convert_universal(self):
        if self.pos in mystem_tags:
            self.pos = mystem_tags[self.pos]
        else:
            self.pos = 'X' # Х - universal тег для неизвестных слов
        return self
    
    # нужна в двух случаях: для Замен, полученных из словарей, чтобы приписывать им соответствующий параметр,
    # и для подсчета адекватности слова контексту
    def cos_sim(self, context):
        #print(self.lexem, target.lexem, self.pos, target.pos)
        subst_query = str(self.lexem+'_'+self.pos)
        target_query = str(context.lexem+'_'+context.pos)
        if subst_query in model and target_query in model:
            return model.similarity(subst_query, target_query)
        else:
            return 0.0

In [19]:
class Substitution(Token):
    def __init__(self, w):
        super().__init__(w)
        self.similarity = None
        self.fitness = None
        self.closeness = None
    
    # для слов из словаря и тезауруса: определяем тег пайморфи, переводим в формат universal - так быстрее, чем майстемом
    def tagging(self, w):
        tag = morph.parse(w)[0].tag.POS
        if tag in pymorphy_tags:
            return pymorphy_tags[tag]
        else:
            return 'X' # Х - universal тег для неизвестных слов
    
    # приписываем недостающие атрибуты словам, взятым из словаря или тезауруса
    def setting_atr(self, target):
        self.pos = self.tagging(self.lexem)
        self.complexity_params(complexity_type)
        self.similarity = self.cos_sim(target)

In [20]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
        self.place = None
        self.context = None
        self.easier = []
        
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                # при этом превращаем их в объекты соответствующего класса
                syn_tokens = []
                for syn, sim in model.most_similar(positive=[query]):
                    syn_text = syn[:syn.find('_')] # текст до части речи
                    
                    syn_tok = Substitution(syn_text) # из текстовой строки инициализируем объект класса
                    syn_tok.pos = syn[syn_tok.len+1:] # часть речи
                    
                    syn_tok.complexity_params(complexity_type) # сложность по функции в зависимости от выбранного параметра
                    
                    syn_tok.similarity = sim # а близость по параметру модели
                    
                    syn_tokens.append(syn_tok)
                    
                return syn_tokens
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target.lexem in cur_line:
                            del(reader)
                            for c in cur_line:
                                if ' ' not in c and c!=target.lexem: # формируем список однословных синонимов
                                    sub_tok = Substitution(c)
                                    
                                    sub_tok.setting_atr(target)
                                    
                                    if sub_tok not in lst:
                                        lst.append(sub_tok) 
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
                #print(lst)
                return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target.lexem in asis:
                lst = []
                for s in asis[target.lexem]:
                    if ' ' not in s: # формируем список однословных синонимов
                        sub_tok = Substitution(s)
                        sub_tok.setting_atr(target)
                        lst.append(sub_tok)
                return lst
            else:
                return []

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self)
        
        if base_type == 'asis':
            self.substituts = asis_search(self)
            
        return self
    
    def find_easier(self, use_min = use_min, threshold = global_threshold):
        for sub in self.substituts:
            if not sub.is_complex(threshold = threshold, use_min = use_min):
                if sub.complexity > self.complexity:
                    self.easier.append(sub)
        return self
    
    def make_window(self, tokens, window = 10):
        context = [self]
        left_ind = self.num-1
        right_ind = self.num+1
        ind = 0
        # добавляем по одному слову слева и/или справа, пока не наберется window + само слово
        while len(context)<window+1:
            while left_ind >= 0:
                left_w = tokens[left_ind]
                left_ind-=1
                # проверка, что это слово
                if any(exception in left_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context[:0] = [left_w] #вставляем слово слева от цепочки
                    ind+=1 # индекс слова сдвигается
                    break

            while right_ind < len(tokens):
                right_w = tokens[right_ind]
                right_ind+=1
                # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
                if any(exception in right_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context.append(right_w) # справа от цепочки
                    break
        self.place = ind
        self.context = context
        return self

## Функция анализа текста и формирования из него списка токенов с соотв.атрибутами. Вход: текст в виде строки

In [21]:
#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        token.convert_universal() # превращаем POS в universal формат
        #print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

## Функция для нахождения сложных слов, превращения их в подкласс сложных слов и поиска замен для них. Вход: список объектов класса Токен

In [61]:
def selecting_complex(tokens, base_type=['yarn','model','asis'][0], threshold = global_threshold, use_min = use_min):
    complex_words = []
    actualy_complex = []
    to_simplify = []
    for token in tokens:
        if token.is_complex(threshold, use_min):
            comp_token = Complex_word(token) # токен становится Сложным словом
            
            comp_token.search_substituts(base_type=base_type)
            
            complex_words.append(comp_token)
            
            # код для принтов
            '''
            if comp_token.substituts:
                for syn in comp_token.substituts:
                    print (comp_token.lexem, comp_token.complexity, base_type, ':', syn.lexem, syn.complexity, syn.similarity)
            '''
            if comp_token.substituts:
                actualy_complex.append(comp_token)
                comp_token.find_easier(threshold, use_min)
                if comp_token.easier:
                    to_simplify.append(comp_token)
                    
            '''if comp_token.easier:
                for syn in comp_token.easier:
                    print (comp_token.lexem, comp_token.pos, base_type, ':', syn.lexem, syn.pos, syn.complexity, syn.similarity)
            '''
        else:
            complex_words.append(token)
    return complex_words, actualy_complex, to_simplify

## Вероятность контекстного окна на основе логарифма + Лапласса. Контекстное окно: 2n+1 слов (по n слева и справа), можно задавать. 

In [23]:
ngrams_dict = {0: {}, 1: unigrams, 2: bigrams, 3: trigrams}

### Вообще при н=1 должно быть не деление на V, а деление на объем корпуса - количество вхождений слов. Те подсчитать сумму значений частот униграмм. Но пока для 3грамм работает нормально. Также криво, вроде бы, с логарифмом... 

In [25]:
def ngram_prob(ngram, n, use_log= False):
    c1 = ' '.join(ngram[-n:]) # в числителе частота строки длины n
    c2 = ' '.join(ngram[-n:-1]) # в знаменателе - строки без последнего символа
    d = ngrams_dict.get(n) # для поиска числителя берем словарь n-грамм
    d2 = ngrams_dict.get(n-1) # для поиска знаменателя - словарь n-1-грамм
    V = len(d)
    #len(ngrams_dict.get(n-1,len(unigrams))) # сглаживание лапласса: добавляем размер словаря знаменателя
    p1 = d.get(c1,0)+1
    p2 = d2.get(c2,0)+V
    
    result =  p1/p2
    
    #print('\t',c1, '/', c2, ':' , p1-1, '/', p2-V, '(', p1, '/', p2, ')', '=', result)
    
    if use_log:
        return math.log(p1)-math.log(p2)
    else:
        return p1/p2
    
def context_prob(ngrams, use_log= False):
    if use_log:
        p_context = 0.0
    else:
        p_context = 1.0
    for ngram in ngrams:
        p = ngram_prob(ngram, 3, use_log= use_log)
        #print(ngram, p)
        if use_log:
            p_context+=p
        else:
            p_context*=p
    return p_context

## Загрузка датафрейма с текстами и измерение потенциала системы для разных словарных баз

In [26]:
df = pd.read_csv('texts1.csv', sep='\t', encoding='utf-8')

In [27]:
df = df.drop(['Unnamed: 0'],axis=1)
df = df.drop_duplicates(subset='texts')

In [28]:
rubrics = list(set(df['rubrics']))
rubrics

['politics',
 'culture',
 'sport',
 'incident',
 'society',
 'business',
 'science',
 'computers',
 'world']

In [37]:
def make_sample(n=15, mode=['random','first'][0], lst = None):
    rubric = list(set(df['rubrics'])) # список рубрик
    text_rubrics = df.filter(items=['texts', 'rubrics']) # таблица текст-рубрика
    text_for_potential = []
    for r in rubric:
        rub_texts = []
        for l in range(len(text_rubrics)):
            if text_rubrics.loc[l][1]==r:
                rub_texts.append(text_rubrics.loc[l][0])
                if mode == 'first':
                    if len(rub_texts)>n-1:
                        break
        if mode == 'random':
            rub_texts = [rub_texts[l] for l in lst] # делаем подвыборку в н текстов
            
        text_for_potential.extend(rub_texts) 
    return text_for_potential

In [62]:
def measure_potential(n = 5, base = ['yarn','model','asis'][0], use_min = False):
    
    text_for_potential = make_sample(n)
    #print(text_for_potential)
    
    positif = 0
    total1 = 0
    #total2 = 0
    
    for text in text_for_potential:
        tokens = text_structuring(text, complexity_type, global_threshold, use_min)        
        complex_words, actually_complex, comlex_easier = selecting_complex(tokens, base, threshold = global_threshold, use_min = use_min)
        '''for token in actually_complex:
            print(token.num, token.text, token.lexem, token.complexity, token.easier)'''
        positif += len(comlex_easier)
        total1 += len(actually_complex)
        
        '''positif += sum([sum([1 for sub in token.easier if sub in minimum]) for token in comlex_easier])
        total1 += len(actually_complex)
        total2 += len(comlex_easier)
        
    pot1 = positif/total1 if total1 else 0
    pot2 = positif/total2 if total2 else 0
    
    return pot1, pot2'''
        
    pot1 = positif/total1 if total1 else 0
    
    return pot1

In [ ]:
lst = random.sample(range(15),5)
model_potential = measure_potential(n = 5, base = 'model', mode = 'random', lst=lst)
model_potential

In [ ]:
asis_potential = measure_potential(n = 5, base = 'asis', mode = 'random', lst=lst)
asis_potential_total

In [ ]:
yarn_potential = measure_potential(n = 5, base = 'yarn', mode = 'random', lst=lst)
yarn_potential_total

## Большая и страшная функция для сложных токенов в тексте, вычисляющая для них всяческие параметры

In [137]:
class_df1 = pd.DataFrame(columns=['исходное','замена','индекс','контекст','сложность','разница_сложности',
                                 'словарность', 'близость','разница_вероятности','разница_лог_вероятности',
                                 'контекстуальность','класс'])
class_df1

,исходное,замена,индекс,контекст,сложность,разница_сложности,словарность,близость,разница_вероятности,разница_лог_вероятности,контекстуальность,класс


In [138]:
def making_frame(window=5, base = ['yarn','model','asis'][0], n=3, t=5, algorithm = ['ngram','random_three'][0], baseline=0):
    # алогритм классификации слова как сложного/простого
    global class_df
    
    def ngram_algorithm(best_sub):
        if best_sub:
            if sub.fitness > best_sub.fitness:
                return sub
            elif sub.fitness == best_sub.fitness:
                if sub.complexity - best_sub.complexity > baseline: 
                    # принимаю условие, что вероятность должна быть больше исходной (baseline=0) 
                    # можно модифицировать на "вероятность не более чем на _ меньше исходной (baseline=-0.01, например)
                    return sub
            else:
                return best_sub
        else:
            return best_sub
            
    def transpose(matr):
        res=[]
        n=len(matr)
        m=len(matr[0])
        for j in range(m):
            tmp=[]
            for i in range(n):
                tmp=tmp+[matr[i][j]]
            res=res+[tmp]
        return res
            
    for text in make_sample(t, mode='first'):
        #print(text)
        tokens = text_structuring(text, complexity_type, global_threshold, use_min)        
        complex_words, actually_complex,complex_easier = selecting_complex(tokens, base, threshold = global_threshold, use_min = False)
        pr = [token.lexem for token in actually_complex]
        #print(pr)
        pr = [token.lexem for token in complex_easier]
        #print(pr)
        for token in complex_easier:
            #['лемма','индекс','контекст','информативность','разница_сложности','словарность',
            # 'близость','разница_вероятности','разница_лог_вероятности','контекстуальность','класс']
            
            new_line = [[]]*12 #будущая часть датафрейма
            #print(len(new_line))
            new_line[0] = [token.lexem]*len(token.easier) # исходное слово
            cls = [0]*len(token.easier) # для таблички - классы
            in_min = [] # для таблички - словарность - в минимуме
            easier_lexems = [t.lexem for t in token.easier] # для таблички - лексемы замен 1
            new_line[1] = easier_lexems
            easier_inf = [t.complexity for t in token.easier] # для таблички - сложность замен 3
            easier_compl = [t.complexity-token.complexity for t in token.easier] # для таблички - разница сложности 4
            new_line[4] = easier_inf
            new_line[5] = easier_compl
            easier_cos = [sub.cos_sim(token) for sub in token.easier] # для таблички - косинусная близость 6
            new_line[7] = easier_cos
            

            # окна контекста строим для тех сложных слов, для которых есть варианты замен проще, чем исходное
            token.make_window(complex_words, window = window) # для подсчетов
            easier_places = [token.place]*len(token.easier) # для таблички - место исходного в контексте 1
            new_line[2] = easier_places

            context_lem = [c.lexem for c in token.context] # для подсчета вероятностей нграмм
            context_text = ' '.join([c.text.lower() for c in token.context]) # для таблички - контекст 2
            new_line[3] = [context_text]*len(token.easier)

            # определение, насколько слово близко с другими словами из контекста
            # cos_sim возвращает 0.0, если пара слов не найдена в модели
            closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])

            text_ngrams = [g for g in ngrams(context_lem, n)] # генерация n-грамм
            p_context = context_prob(text_ngrams, use_log=False) # вероятность контекста
            p_context_log = context_prob(text_ngrams, use_log=True) # вероятность контекста

            
            best_sub = None
            easier_prob = [] # для таблички - контексная вероятность 7
            easier_prob_log = [] # для таблички - log контексная вероятность 8
            easier_similarity = [] # для таблички - контексная близость 9
            for sub in token.easier:
                
                # контекст с новым словом
                sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                # все то же самое
                sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
                easier_similarity.append(sub.closeness) # для таблички - контексная близость

                sub_context_lem = [t.lexem for t in sub_context]
                sub_ngrams = [g for g in ngrams(sub_context_lem, n)]
                p_changed_context = context_prob(sub_ngrams, use_log=False)-p_context
                easier_prob.append(p_changed_context) # для таблички - контексная вероятность
                
                p_changed_context2 = context_prob(sub_ngrams, use_log=True)-p_context_log
                sub.fitness = p_changed_context2
                easier_prob_log.append(p_changed_context2) # для таблички - log контексная вероятность

                if sub in minimum:
                    in_min.append(1) # словарность
                else:
                    in_min.append(0)

                if algorithm == 'ngram':
                    best_sub = ngram_algorithm(best_sub)
                    #print('Пока лучший ',best_sub, p_changed_context2)
            
            new_line[6] = in_min # словарность
            new_line[8] = easier_prob # вероятность
            new_line[9] = easier_prob_log
            new_line[10] = easier_similarity # контекстуальность
            
            if algorithm == 'ngram' and best_sub:
                for i,sub in enumerate(token.easier):
                    if sub == best_sub:
                        cls[i]=1
                        
            new_line[11] = cls # класс
            
            #print(new_line)
            # запись в датафрейм
            new_line = transpose(new_line)
            for line in new_line:
                class_df1.loc[len(class_df1)]=line
            print(new_line)
        
        class_df1 = class_df1.drop_duplicates(subset=['исходное','замена','индекс','контекст'])
        class_df1.to_csv('class_df_yarn.csv',sep='\t')
            
            
            

In [139]:
making_frame(window=6, base = 'yarn', n=3, t=15, algorithm = 'ngram', baseline=0)

UnboundLocalError: local variable 'class_df1' referenced before assignment

In [128]:
len(class_df)

733

In [129]:
class_df

,исходное,замена,индекс,контекст,сложность,разница_сложности,словарность,близость,разница_вероятности,разница_лог_вероятности,контекстуальность,класс
0,мэрия,администрация,0,мэрия екатеринбурга готова провести опрос насе...,-9.758051,2.118905,0,0.488974,-1.765001e-38,-0.000003,0.019178,0
1,опрос,исследование,3,екатеринбурга готова провести опрос населения ...,-8.779933,2.464501,0,0.512625,-7.517182e-37,-0.000148,0.015821,0
2,интерфакс,тасс,3,сквера сообщает агентство интерфакс со ссылкой на,-12.146207,1.029027,0,0.688722,2.041090e-35,0.000016,0.060943,0
3,мэр,депутат,3,ссылкой на заместителя мэра города екатерину к...,-9.544944,1.092480,0,0.374717,6.614308e-37,0.000130,0.028308,0
4,опрос,исследование,3,предложением о проведении опроса местных жител...,-8.779933,2.464501,0,0.512625,-2.282582e-31,-1.386323,-0.032981,0
5,спрос,цена,3,должен быть особый спрос а они обязаны,-8.694857,1.759700,0,0.462627,-7.467853e-36,-0.000067,-0.005002,0
6,спрос,рынок,3,должен быть особый спрос а они обязаны,-8.791276,1.663281,0,0.396855,-7.816035e-36,-0.000070,-0.002635,0
7,обязывать,должный,3,спрос а они обязаны понимать что несут,-7.060074,4.098417,0,0.434679,2.765546e-27,11.825139,-0.020627,0
8,присяга,решение,3,что приведение к присяге избранного президента...,-8.152354,3.160901,0,0.349902,-2.538559e-32,-1.792054,-0.011637,0
9,присяга,конституция,3,что приведение к присяге избранного президента...,-10.270540,1.042715,0,0.347395,-2.538382e-32,-1.791705,0.080414,0


In [120]:
class_df.to_csv('class_df.csv',sep='\t', encoding = 'utf-8')

PermissionError: [Errno 13] Permission denied: 'class_df.csv'

## Штука. Пока не трогать

In [67]:
for token in complex_words:
    if isinstance(token, Complex_word):
        if token.easier: 
            # окна контекста строим для тех сложных слов, для которых есть варианты замен, которые проще, чем слово
            token.make_window(complex_words, window = 5)
            
            context_lem = [c.lexem for c in token.context]
            
            # определение, насколько слово близко с другими словами из контекста
            closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])
            
            print(token.text, token.place, ':', context_lem)
            
            # генерация n-грамм
            text_3grams = [n for n in ngrams(context_lem, 3)]
            
            print(text_3grams)
            
            p_context = context_prob(text_3grams, use_log=True)
                
            print('\nВероятность слова _{0}_ в контексте {1} = {2}, близость к контексту = {3}\n'.format(token.text, context_lem, p_context,closeness))
            
            best_fitness = -1000
            best_sub = None
            
            for sub in token.easier:
                
                # контекст с новым словом
                sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                
                # определение, насколько слово близко с другими словами из контекста
                sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
            
                sub_context_lem = [t.lexem for t in sub_context]
                
                sub_3grams = [n for n in ngrams(sub_context_lem, 3)]
                
                p_changed_context = context_prob(sub_3grams, use_log=True)
                
                fit='НЕТ'
                if p_changed_context>p_context:
                    fit='ДА'
                    sub.fitness = p_changed_context
                    if sub.fitness > best_fitness:
                        best_fitness = sub.fitness
                        best_sub = sub
                    elif sub.fitness == best_fitness:
                        if best_sub.complexity < sub.complexity:
                            best_fitness = sub.fitness
                            best_sub = sub
                        
                print('\nВероятность замены _{0}_ в контексте {1} = {2}, разница = {3}, используем? - {4}'.format(sub.text, sub_context_lem, p_changed_context, p_context-p_changed_context, fit))
                print('Близость к контексту = {0}, разница с исходным = {1}\n'.format(sub.closeness, closeness-sub.closeness))
            if best_sub:
                print('Лучшее значение вероятности: {0}\n'.format(best_sub.text))
            

изъяли 3 : ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный']
[('бракованный', 'белорусский', 'лекарство'), ('белорусский', 'лекарство', 'изымать'), ('лекарство', 'изымать', 'из'), ('изымать', 'из', 'обращение'), ('из', 'обращение', 'лекарственный')]
	 бракованный белорусский лекарство / бракованный белорусский : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 белорусский лекарство изымать / белорусский лекарство : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 лекарство изымать из / лекарство изымать : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 изымать из обращение / изымать из : 26 / 450 ( 27 / 2876993 ) = 9.38479864219343e-06
	 из обращение лекарственный / из обращение : 0 / 158 ( 1 / 2876701 ) = 3.4762041658135483e-07

Вероятность слова _изъяли_ в контексте ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный'] = -71.0648734045552, близость к контексту = 0.06318795363628794

	 бракованный белорусский 

поправить вычисление вероятности - в знаменателе число уникальных нграмм, а не униграмм, ну и переписать с условием n>/=1
изменить вычисление вероятности для логарифма
Написать перплексию для получившейся модели - хз как????  
поправить нахождение best_sub - если их больше 1, надо как-то это обрабатывать  
прогнать статистическую модель на текстах и сформировать таким образом корпус положительных замен, которые потом можно подредачить ручками   
заодно при редактировании ручками можно создать еще один файл, в котором редачить, а потом сравнить с первым, чтобы посчитать точность и проч  
потом все-таки посчитать 

In [ ]:
def perplexity(docs):
    docs = np.asarray(docs)
    out_of_dict = []
   
    tmp_sum_docs = 0.0
    N = 0.0
    for doc in log_progress(docs):
        tmp_sum_words = 0.0
        for word in log_progress(words):
            freq = frequencies.get(word, TOL)
            freq_empirical = frequencies_empirical.get(word, 0.0)
            N += freq_empirical
            if freq == TOL:
                out_of_dict.append(word)
            tmp_sum_words += freq_empirical * np.log(freq)
        tmp_sum_docs += tmp_sum_words
       
    return np.exp(-tmp_sum_docs / N), out_of_dict